In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nasdaq/nasdaq_screener_1734427367061.csv


In [3]:
!pip install --upgrade yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 17.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp310-cp310-linux_x86_64.whl size=317853 sha256=e1c3f1a0345c548e2e88789f6fbcc5d44be1d73babe86e0e9038bdd50fa32a76
  Stored in directory: /root/.cache/pip/wheels/75/79/e5/8838db0594cc6c587142fd2563356392ade6255c5930411069
Successfully built peewee


In [4]:
import pandas as pd
import yfinance as yf

# Шаг 1: Загрузка исходного файла с данными о компаниях NASDAQ
input_file = '/kaggle/input/nasdaq/nasdaq_screener_1734427367061.csv'
output_file = 'nasdaq_max_historical_updated.xlsx'

# Чтение данных из CSV
nasdaq_df = pd.read_csv(input_file)

# Проверяем, есть ли колонка "Symbol" для тикеров
if 'Symbol' not in nasdaq_df.columns:
    raise ValueError("В файле нет колонки 'Symbol', необходимой для загрузки данных.")

# Создаем новый DataFrame для хранения максимальных значений
max_values = []

# Шаг 2: Получение исторических данных по каждому символу
for ticker in nasdaq_df['Symbol']:
    try:
        print(f"Загружаем данные для {ticker}...")
        data = yf.download(ticker, start="1990-01-01", end="2024-12-31", progress=False)
        
        # Рассчитываем максимальные значения по годам
        data['Year'] = data.index.year
        yearly_max = data.groupby('Year')['High'].max().reset_index()
        yearly_max.columns = ['Year', 'Max_High']
        
        # Добавляем тикер для идентификации
        yearly_max['Symbol'] = ticker
        max_values.append(yearly_max)
    except Exception as e:
        print(f"Ошибка при загрузке данных для {ticker}: {e}")
        continue

# Объединяем данные по всем тикерам
max_values_df = pd.concat(max_values, ignore_index=True)

# Шаг 3: Добавляем полученные данные в конец исходного файла
with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
    nasdaq_df.to_excel(writer, index=False, sheet_name='Company_Info')
    max_values_df.to_excel(writer, index=False, sheet_name='Max_Historical_Highs')

print(f"Файл успешно обновлен и сохранен как '{output_file}'.")


Загружаем данные для A...
Загружаем данные для AA...
Загружаем данные для AACG...
Загружаем данные для AACT...
Загружаем данные для AADI...
Загружаем данные для AAL...
Загружаем данные для AAM...
Загружаем данные для AAME...
Загружаем данные для AAOI...
Загружаем данные для AAON...
Загружаем данные для AAP...
Загружаем данные для AAPL...
Загружаем данные для AAT...
Загружаем данные для AB...
Загружаем данные для ABAT...
Загружаем данные для ABBV...
Загружаем данные для ABCB...
Загружаем данные для ABCL...
Загружаем данные для ABEO...
Загружаем данные для ABEV...
Загружаем данные для ABG...
Загружаем данные для ABL...
Загружаем данные для ABLLL...
Загружаем данные для ABLLW...
Загружаем данные для ABLV...
Загружаем данные для ABLVW...
Загружаем данные для ABM...
Загружаем данные для ABNB...
Загружаем данные для ABOS...
Загружаем данные для ABP...
Загружаем данные для ABR...
Загружаем данные для ABR^D...
Загружаем данные для ABR^E...
Загружаем данные для ABR^F...
Загружаем данные для ABS

In [ ]:
import pandas as pd
import yfinance as yf

# Шаг 1: Загрузка исходного файла с компаниями NASDAQ
input_file = '/kaggle/input/nasdaq/nasdaq_screener_1734427367061.csv'  # Путь к вашему файлу
output_file = 'nasdaq_with_historical_data.xlsx'  # Выходной файл

# Загружаем файл с компаниями NASDAQ
companies_df = pd.read_csv(input_file)

# Оставляем только нужные столбцы
companies_df = companies_df[['Symbol', 'Name', 'Country', 'IPO Year', 'Sector', 'Industry']]

# Шаг 2: Функция для получения цены на заданную дату
def get_price_on_date(ticker, year, month=12, day=1):
    try:
        date = f"{year}-{month:02d}-{day:02d}"
        stock = yf.Ticker(ticker)
        data = stock.history(start=date, end=date)
        if not data.empty:
            return data['Close'].iloc[0]
        else:
            return None
    except Exception as e:
        print(f"Ошибка для {ticker} на {date}: {e}")
        return None

# Шаг 3: Формирование данных по годам
years = list(range(1990, 2025))  # Укажите диапазон лет
historical_data = []

for index, row in companies_df.iterrows():
    ticker = row['Symbol']
    print(f"Обработка: {ticker} ({index + 1}/{len(companies_df)})")
    company_data = {'Symbol': ticker, 'Name': row['Name'], 'Country': row['Country'],
                    'IPO Year': row['IPO Year'], 'Sector': row['Sector'], 'Industry': row['Industry']}
    
    for year in years:
        price = get_price_on_date(ticker, year)
        company_data[year] = price
    
    historical_data.append(company_data)

# Шаг 4: Сохранение результата в Excel
result_df = pd.DataFrame(historical_data)
result_df.to_excel(output_file, index=False)

print(f"Файл успешно сохранен: {output_file}")


Обработка: A (1/7000)
Обработка: AA (2/7000)
Обработка: AACG (3/7000)
Обработка: AACT (4/7000)
Обработка: AADI (5/7000)
Обработка: AAL (6/7000)
Обработка: AAM (7/7000)
Обработка: AAME (8/7000)
Обработка: AAOI (9/7000)
Обработка: AAON (10/7000)
Обработка: AAP (11/7000)
Обработка: AAPL (12/7000)
Обработка: AAT (13/7000)
Обработка: AB (14/7000)
Обработка: ABAT (15/7000)
Обработка: ABBV (16/7000)
Обработка: ABCB (17/7000)
Обработка: ABCL (18/7000)
Обработка: ABEO (19/7000)
Обработка: ABEV (20/7000)
Обработка: ABG (21/7000)
Обработка: ABL (22/7000)
Обработка: ABLLL (23/7000)
Обработка: ABLLW (24/7000)
Обработка: ABLV (25/7000)
Обработка: ABLVW (26/7000)
Обработка: ABM (27/7000)
Обработка: ABNB (28/7000)
Обработка: ABOS (29/7000)
Обработка: ABP (30/7000)
Обработка: ABR (31/7000)
Обработка: ABR^D (32/7000)
Обработка: ABR^E (33/7000)
Обработка: ABR^F (34/7000)
Обработка: ABSI (35/7000)
Обработка: ABT (36/7000)
Обработка: ABTS (37/7000)
Обработка: ABUS (38/7000)
Обработка: ABVC (39/7000)
Обрабо